<a href="https://colab.research.google.com/github/utsavdutta/recommender_system/blob/master/ranking_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [148]:
import numpy as np
from typing import Dict, Text
import tensorflow as tf
import pprint
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import os
import pprint
import tempfile

In [149]:
ratings = tfds.load('movielens/100k-ratings', split="train");
ratings = ratings.map(lambda x: {
     "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
});

for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x);

# print();
# movies = tfds.load('movielens/100k-movies', split="train")
# for x in movies.take(1).as_numpy_iterator():
#   pprint.pprint(x);

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'user_id': b'138',
 'user_rating': 4.0}


In [150]:
tf.random.set_seed(42);
shuffle = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False);
train = shuffle.take(80_000);
test = shuffle.skip(80_000).take(20_000);

In [151]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"]);
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"]);
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)));
unique_user_ids = np.unique(np.concatenate(list(user_ids)));

In [154]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

TypeError: ignored

In [137]:
class MovieLensModel(tfrs.models.Model):
    def __init__(self):
      super().__init__()
      self.ranking_model: tf.keras.Model = RankingModel();
      self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
              loss= tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()]);

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
       rating_predictions = self.ranking_model(features["user_id"], features["movie_titles"]);
       return self.task(lables = features["user_rating"], predictions = rating_predictions);

In [ ]:
model = MovieLensModel();
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1));
cached_train = train.shuffle(100_000).batch(8192).cache();
cached_test = test.batch(4096).cache();
model.fit(cached_train, epochs=3);